In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(5)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

# = = =  = = =  = = =

import pandas as pd

# = = =  = = =  = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

import time

# = = =  = = =  = = =

with open('./api.txt', 'r') as api_txt:
    api = api_txt.read()

# = = =  = = =  = = =

print('Data Loading...')
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel('./file/1.exhibitor.xlsx',
                                                                       header=0,
                                                                       dtype=str).fillna('')
print('Loading Done !')
print()

df_input['No'] = df_input['No'].astype(int)

crawler_remain = len(df_input)

print(f'总数量：{crawler_remain}')
print()

# = = =  = = =  = = =

def crawler(crawler_tuple):
    global api, list_dict_correct, list_series_error, crawler_remain

    crawler_status, (crawler_index, crawler_series) = False, crawler_tuple

    # = = =  = = =  = = =

    try:
        crawler_retry = 0
        while True:
            crawler_retry += 1

            try:
                url_request, url_check = f'''https://api.messefrankfurt.com/service/esb_api/exhibitor-service/api/2.1/public/exhibitor/profile/zh-CN/{crawler_series['Url'].removeprefix('https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/').removesuffix('.html')}/AUTOMECHANIKASHANGHAI''', crawler_series['Url']

                resp = requests.get(url_request,
                                    headers=get_header(**{'Apikey': api}),
                                    proxies=get_proxy(),
                                    timeout=(5, 15))

                if resp.status_code == 200 or resp.status_code == 401:
                    dict_ = resp.json()

                    if resp.status_code == 200 and dict_['success']:
                        break
                    elif resp.status_code == 401 and dict_['status'] == 401 and dict_['message'].strip() == 'API GW: Unauthorized':
                        print(f'fuck api {api}')
                        time.sleep(3)
                        with open('./api.txt', 'r') as api_txt:
                            api = api_txt.read()
            except KeyboardInterrupt:
                break
            except:
                continue

        # = = =  = = =  = = =

        list_dict_correct_temp = []
        if dict_['result']['products']['countTotal'] == 0:
            list_dict_correct_temp.append({'Url': crawler_series['Url']})
        else:
            list_dict_correct_temp = []
            for i, dict_product in enumerate(dict_['result']['products']['products']):
                list_dict_correct_temp.append({'Url': crawler_series['Url'],
                                               '[Product] No': i + 1,
                                               '[Product] Url': f'''{crawler_series['Url'].removesuffix('.html')}/{dict_product['id']}.html'''})

        # = = =  = = =  = = =

        with semaphore_correct:
            list_dict_correct.extend(list_dict_correct_temp)

        # = = =  = = =  = = =

        crawler_status = True
    except:
        crawler_series['Check_Url'] = url_check

        with semaphore_error:
            list_series_error.append(crawler_series)

    # = = =  = = =  = = =

    if crawler_remain % 30 == 0:
        clear_output()
    print(f'''[{crawler_status}] - [请求{crawler_retry}次] - [剩余{crawler_remain - 1}条] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['No']}. {url_check}\n''')

    with semaphore_remain:
        crawler_remain -= 1

# = = = = = = = = = = = = = = =

pool.map(crawler, df_input.iterrows())

print('Data Outputting...')
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Url', '[Product] No'],
                                                             ascending=[True, True],
                                                             ignore_index=True)
    df_correct.to_excel(f'''./file/2.product_url-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
print('Output Finished !')
print()
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./file/product_url_error.xlsx', index=False)
    print('- Error Occurred -')
    print()

# = = =  = = =  = = =

print('Done ~')

[True] - [请求2次] - [剩余29条] - [2024/12/02 14:34:44] - 6222. https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/zhongshan-city-ninglang-electronictechnology-co-ltd.html

[True] - [请求1次] - [剩余28条] - [2024/12/02 14:34:44] - 6304. https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/zhu-ji-yibo-auto-tool-factory.html

[True] - [请求1次] - [剩余27条] - [2024/12/02 14:34:44] - 6302. https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/zhuji-xiongliang-autoparts-co-ltd.html

[True] - [请求1次] - [剩余26条] - [2024/12/02 14:34:45] - 6306. https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/zhuji-zenghui-machinery-co-ltd.html

[True] - [请求1次] - [剩余25条] - [2024/12/02 14:34:45] - 6305. https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/zhuji-yisen-machinery-technology-co-ltd.html

[True] -